In [15]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
from keras import backend as K


import pandas as pd
import numpy as np
import codecs
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
X_train_Q1 = pd.read_excel('../data/interim/X_train_Q1_clean.xlsx')
X_valid_Q1 = pd.read_excel('../data/interim/X_valid_Q1_clean.xlsx')

y_train_Q1 = pd.read_excel('../data/interim/y_train_Q1.xlsx')
y_valid_Q1 = pd.read_excel('../data/interim/y_valid_Q1.xlsx')

In [3]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
# function adapted from `preprocess` function of lab 3 
# of this course (575 Advance Learning Machine)
def preprocess_comments(text, 
                        min_token_len = 2, 
                        irrelevant_pos = ['PRON', 'SPACE', 'PUNCT', 'ADV', 
                                          'ADP', 'CCONJ', 'AUX', 'PRP'],
                        avoid_entities = ['PERSON', 'ORG', 'LOC', 'GPE']):
# note: Didn't use the following options in the `preprocess_comments`...
#    - 'PROPN' because it erases proper names as 'George', but also words as orange.
#    - 'DET' since it removes the word 'no', which changes the meaning of a sentence.
# *for more information see link: https://universaldependencies.org/u/pos/
    """
    Given text, min_token_len, irrelevant_pos and avoid_entities, carries out 
    preprocessing of the text and returns list of preprocessed text. 
    Parameters
    -------------
    text : (list) 
        the list of text to be preprocessed
    min_token_len : (int) 
        min_token_length required
    irrelevant_pos : (list) 
        a list of irrelevant pos tags
    avoid_entities : (list)
        a list of entity labels to be avoided
    Returns
    -------------
    (list) list of preprocessed text
    Example
    -------------
    >>> example = ["Hello, I'm George and I love swimming!",
                   "I am a really good cook; what about you?",
                   "Contact me at george23@gmail.com"]
    >>> preprocess(example)
    (output:) ['hello love swimming', 'good cook', 'contact']
    """
    result = []
    others = ["'s", "the", "that", "this", "to", "-PRON-"]
    # I add "-PRON-" that erase "my", "your", etc. other way to erase them is to
    #   use adding 'DET' to irrelevant_pos but it would erase the word 'no' too.
    for sent in text:
        sent = sent.lower()
        sent = re.sub(r"facebook", "social media", sent)
        sent = re.sub(r"twitter", "social media", sent)
        sent = re.sub(r"instagram", "social media", sent)
        sent = re.sub(r"whatsapp", "social media", sent)
        sent = re.sub(r"linkedin", "social media", sent)
        sent = re.sub(r"snapchat", "social media", sent)
        result_sent = []
        doc = nlp(sent)
        entities = [str(ent) for ent in doc.ents if ent.label_ in avoid_entities]
        # This helps to detect names of persons, organization and dates
        for token in doc:            
#             if (token.is_stop == True or
            if(token.like_email or
                token.like_url or
                token.pos_ in irrelevant_pos or
                str(token) in entities or
                str(token.lemma_) in others or
                len(token) < min_token_len):
                continue
            else:
                result_sent.append(token.lemma_)
        result.append(" ".join(result_sent))
    return result

In [ ]:
X_valid = pd.read_csv('data/X_valid.csv')

preprocessed_x_train = preprocessing.preprocess_comments(X_train['Comment'])
preprocessed_x_valid = preprocessing.preprocess_comments(X_valid['Comment'])


X_train_ppd = pd.DataFrame(preprocessed_x_train, columns=['Comment'])
X_valid_ppd = pd.DataFrame(preprocessed_x_valid, columns=['Comment'])

In [4]:
df = pd.concat([X_train_Q1, y_train_Q1.iloc[:,:12]], axis = 1)

In [5]:
df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0


In [10]:
X_train_ppd = pd.DataFrame(preprocess_comments(df['Comment']), columns=['Comment'])
X_valid_ppd = pd.DataFrame(preprocess_comments(df['Comment']), columns=['Comment'])

In [11]:
X_train_ppd.head()

,Comment
0,real diversity need create seat table mean aff...
1,keep building warm provide warm water bathroom
2,well communication top
3,would beneficial if management not micro manage
4,more education applicable job


In [12]:
def max_len(x):
    a=x.split()
    return len(a)
max_len = max(X_train_ppd['Comment'].apply(max_len))
max_len

87

In [13]:
vect=Tokenizer()
vect.fit_on_texts(X_train_ppd['Comment'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

8638


In [16]:
embeddings_matrix = {}
f = codecs.open('/Users/karan/Downloads/crawl-300d-2M.vec', encoding='utf-8')
for line in f:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_matrix[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_matrix))

found 1999996 word vectors


In [17]:
# create a weight matrix for words in training docs
embedding_matrix_fastext = np.zeros((vocab_size, 300))
for word, i in vect.word_index.items():
    embedding_vector = embeddings_matrix.get(word)
    if embedding_vector is not None:
        embedding_matrix_fastext[i] = embedding_vector

In [18]:
np.save('../models/embedding_matrix_fastext_ppd', embedding_matrix_fastext)